# Final Project

## Data Exploration

In [2]:
from xml.dom.minidom import parse
from math import log
import xml
import os
import sys
import csv 
import json
import re
import requests
import math
import xml.dom.minidom
from collections import Counter
data_path = r"C:\Programming\Python\Programs_NLP\wikihowcom-20141208-current.xml"
if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

batches = []
batch_size = 100
batch_count = 0
lines = 12328007

In [18]:
def process_batch(batches):

        with open('temp.xml', 'w') as f:
            f.write('<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.8/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.8/ http://www.mediawiki.org/xml/export-0.8.xsd" version="0.8" xml:lang="en">\n')
            for item in batches:
                f.write("%s" % item.encode('utf8'))
            f.close()

        with open('temp.xml', 'a') as f:
            f.write('</mediawiki>')
            f.close()
        data_dict = parse_data('test.xml')
        keys = [key for key in data_dict]

        posts = []
        for key in keys:
            posts.append(parse_(key, data_dict))

        with open('temp.json', 'w') as outfile:
            for pa in posts:
                json.dump({ "index" : {"_index": "wikihow", "_type": "page", "_id": pa["id"]}}, outfile)
                outfile.write('\n')
                json.dump(pa, outfile)
                outfile.write('\n')
            outfile.close()
        bulk_data = []

        for pa in posts:
            bulk_data.append({"index" : {"_index": "wikihow", "_type": "page", "_id": pa["id"]}})
            bulk_data.append(json.dumps(pa, ensure_ascii=False))

        print("Batch Number: " + str(batch_count) + " Processed")
        print("Indexing Batch Number: " + str(batch_count) + "...")
# 		config = {'host': 'localhost'}
# 		es = elasticsearch.Elasticsearch([config,], timeout=300)
# 		res = es.bulk(index = 'wikihow', body = bulk_data)
        #print(res)
        # check data is in there, and structure in there
        #print("Test Import...")
        #print(json.dumps(es.search(body={"query": {"match_all": {}}}, index = 'wikihow')))

def parse_data(file):
    file_dict = {}
    # Open XML document using minidom parser
    DOMTree = xml.dom.minidom.parse(file)
    mediawiki = DOMTree.documentElement
    #if mediawiki.hasAttribute("xmlns"):
        #print("Root element : %s" % mediawiki.getAttribute("xmlns"))
    # Get all the pages on file
    pages = mediawiki.getElementsByTagName("page")
    # Get high level namespace information.
    for page in pages:
        page_dom = page.getElementsByTagName('text')[0]
        dom_text = page_dom.childNodes[0].data
        if str(dom_text[0]) == '#':
            pass
        else:
            id_ = page.getElementsByTagName('id')[0]
            id_ = id_.childNodes[0].data
            title = page.getElementsByTagName('title')[0]
            title = title.childNodes[0].data
            text = page.getElementsByTagName('text')[0]
            text = text.childNodes[0].data
            file_dict[int(id_)] = [title, text]
    return file_dict


#data_dict = parse_data(data_path)

def structure(page_id, file_dict):
    file_ = file_dict[page_id][1]
    lines = file_.split("\n")
    lines_ = []
    for line in lines:
        if line != "":
            lines_.append(line)
    intro = lines_[0]
    try:
        if "Category" in lines_[1]:
            cat = lines_[1]
            if ":" in cat:
                cat = cat.split(":")[1][:-2]
        else:
            cat = None
    except IndexError:
        cat = None
    try:
        if "Category" in lines[2] and cat is not None:
            cat = cat + "," + re.sub('[^A-Za-z0-9\s\.|\:\*]+', '', lines[2]).split(":")[1]
            process = lines_[3:]
        else:
            process = lines_[2:]
    except IndexError:
        process = None

    return intro, cat, process

def merge(data_list):
    try:
        data_list = " ".join(data_list)
        data_list = data_list.split(" ==")
        data_list = list(zip(*[iter(data_list)] * 2))
    except TypeError:
        data_list = []
    return data_list

def construct_dict(data_list):
    textdict = {}
    steps_dict = {}
    #construct step dictt
    try:
        if data_list[0][1] != "":
            title = re.sub(r'([^\:\w])+', '', data_list[0][0]) 
            #print(title)
            steps = data_list[0][1].split(" #")
            steps = [st for st in steps if st != ""]
            step_index = ["step" + str(number + 1) for number in range(len(steps))]
            step_data = [ste.split("<br><br>") for ste in steps]
            step_new =  []
            for step in step_data:
                step_new.append(step[0])  
            step_data = step_new
            steps_dict = dict(zip(step_index, step_data))
            #return steps_dict
            textdict["steps"] = steps_dict
        subsections = []
        for sect in data_list[1:]:
            if sect == "":
                sect.pop(0)
                #print(sect)
            title = sect[0]
            title = re.sub(r'([^\:\w_])+', '', title)
            title  = "".join([t for t in title if t != " "])
            #print(sect[1:])
            content = sect[1]
            if content[1] == "*":
                subs = content.split("*")
                subs = [s for s in subs if s != " "]
                subs = [re.sub('[^A-Za-z0-9\s\.|\:\*]+', '', string) for string in subs]
                textdict[title] = subs
        return textdict
    except IndexError:
        return {}


def parse_(page_id, data_dict):
    structured = structure(page_id, data_dict)
    final_dict = construct_dict(merge(structured[2]))
    final_dict["title"] = data_dict[page_id][0]
    final_dict["introduction"] = structured[0]
    final_dict["category"] = structured[1]
    final_dict["id"] = page_id
    #parsed = json.loads(json.dumps(final_dict, ensure_ascii=False, sort_keys=True))
    #print(json.dumps(parsed, indent=4, sort_keys=True))
    return final_dict

## Parse Data (this will tax your memory)

In [39]:
data_dict = parse_data(data_path)

In [40]:
len(data_dict)

198150

In [56]:
cat_list = []
for val in data_dict.values():
    cat = re.search(r'(?<=\[\[category:)[^\s\\\]]*',val[1],flags=re.I)
    if cat is not None:
        cat_list.append(cat.group(0))

In [59]:
Counter(cat_list).most_common()

[('Drawing', 2117),
 ('Club', 1587),
 ('Managing', 1585),
 ('Hair', 1513),
 ('Social', 1329),
 ('Animal', 1289),
 ('Making', 1271),
 ('Home', 1268),
 ('School', 1262),
 ('Personal', 1252),
 ('Health', 1223),
 ('Internet', 1114),
 ('Windows', 1074),
 ('Basic', 1041),
 ('Food', 999),
 ('Online', 982),
 ('Website', 981),
 ('Fashion', 950),
 ('Buying', 943),
 ('Dealing', 930),
 ('Video', 917),
 ('Getting', 846),
 ('Youth', 840),
 ('Pokemon', 822),
 ('Improving', 813),
 ('RuneScape', 803),
 ('Personality', 790),
 ('Christmas', 779),
 ('Growing', 768),
 ('Minecraft', 753),
 ('Games', 748),
 ('Music', 747),
 ('Android', 742),
 ('Crushes', 730),
 ('Surviving', 725),
 ('Car', 725),
 ('Creating', 711),
 ('Microsoft', 707),
 ('Business', 705),
 ('Travel', 700),
 ('Halloween', 693),
 ('Cleaning', 693),
 ('Crafts', 680),
 ('Dog', 675),
 ('Skin', 655),
 ('Job', 613),
 ('IPhone', 609),
 ('Sewing', 606),
 ('Bedroom', 606),
 ('Computers', 605),
 ('Dating', 605),
 ('Writing', 591),
 ('Raising', 585),
 (